In [ ]:
from docplex.mp.model import Model
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import os
import warnings
warnings.filterwarnings('ignore')

def euclid_dist (x1, y1, x2, y2):
    return math.sqrt((x1-x2)**2 + (y1-y2)**2)

#input data from .csv file
wh_coords = pd.read_csv("./coordinates_old.csv")
x_coord=wh_coords['x'].values
y_coord=wh_coords['y'].values
node_coords = pd.read_csv("./coordinates.csv")
x_node=node_coords['x_node'].values
y_node=node_coords['y_node'].values
product_data=pd.read_csv("./product_data.csv", index_col=0)
Q=product_data['TOT_MP'].values
W=product_data['VOL_DEPLETING'].values
#MP=product_data['MP'].values 
VOL = product_data['VOL'].values
LC = product_data['#LC'].values
cast = product_data['CAST'].values
internal = product_data['INT'].values
Z = [(350,65),(570,65),(424,65),(60,65),(490,340),(630,225),(60,165),(220,165),(127,165),(375,340),(563,390)]

#node and distances
nodes = {row['node_id']: (row['x_node'], row['y_node']) for _, row in node_coords.iterrows()}
A=[
    (0,3,1),(0,3,13,2),(0,3),(0,3,1,20,4),(0,3,13,2,14,6,15,16,5),(0,3,13,2,14,16),(0,3,1,20,9,7),(0,3,1,20,9,8),(0,3,1,20,9),(0,3,13,19,18,17,10),(0,3,13,2,14,6,15,16,11)
    ]

def calculate_distance(node1_id, node2_id):
    x1, y1 = nodes[node1_id]
    x2, y2 = nodes[node2_id]
    return math.sqrt((x2 - x1)**2 + (y2 - y1)**2)

# Calculate the total distance for each sub-array in A
dis_buffer1 = []
for sub_array in A:
    total_distance = 0
    for i in range(len(sub_array) - 1):
        total_distance += calculate_distance(sub_array[i], sub_array[i + 1])
    dis_buffer1.append(total_distance)

#Input data from context
Class=3 
P=len(product_data)
G=len(cast)
C = 3.5 #Slot max capacity
Zone_1 = range(1, 13)  # Esterno impianto 1
Zone_2 = range(13, 50)  # Esterno impianto 2
Zone_3 = range(50, 173)  # Esterno impianto 3
Zone_4 = range(173, 209)  # Esterno impianto 4
Zone_5 = range(209, 308)  # Esterno SEP 1
Zone_6 = range(308, 348)  # Esterno SEP 2
Zone_7 = range(348, 360)  # Interno SEP 1
Zone_8 = range(360, 401)  # Interno SEP 2
Zone_9 = range(401, 680)  # Interno SEP 3
Zone_10 = range(680, 778)  # Esterno magazzino nuovo 1
Zone_11 = range(778, 828)  # Esterno magazzino nuovo 2
Stock_Zones = [Zone_1, Zone_2, Zone_3, Zone_4, Zone_5, Zone_6, Zone_7, Zone_8, Zone_9, Zone_10, Zone_11]
Cast_Zones = [Zone_1, Zone_2, Zone_3, Zone_4, Zone_5, Zone_6, Zone_7, Zone_8, Zone_9, Zone_11]

dis_buffer2 = []
for i, (zx, zy) in enumerate(Z):  # Iterate over Z with index i
    for k in Stock_Zones[i]:  # Access the corresponding Stock_Zone using index i
        dist = euclid_dist(zx, zy, x_coord[k-1], y_coord[k-1])  # Calculate Euclidean distance
        if x_coord[k-1] > x_coord[0] and x_coord[k-1] < zx:  # Check x-coordinate condition
            dist *= -1  # Multiply distance by -1
        if x_coord[k-1] < x_coord[0] and x_coord[k-1] > zx:  # Check x-coordinate condition
            dist *= -1  # Multiply distance by -1
        dis_buffer2.append(dist)  # Append the adjusted distance

dis = [0] * len(x_coord)
counter = 0
for i in range(len(Z)):
    for j in Stock_Zones[i]:
        dis[j-1] = dis_buffer1[i] + dis_buffer2[counter]
        counter += 1

L=len(dis)

#Apc Matrix
A=np.zeros((P,Class),int) 

product_data['Class']='C' 
for i in range(len(product_data)):
    if W[i]>=15: 
        product_data['Class'][i] ='A' 
        A[[i],0] = 1 
    elif W[i]>=5 and W[i]<15:
        product_data['Class'][i]='B'
        A[[i],1]=1 
    else:
        product_data['Class'][i]='C'
        A[[i],2]=1 

restricted = []
for i in range(len(product_data)):
    if product_data['Class'][i] =='A': 
        restricted.append(i)

mdl = Model(name= 'WH_REVAMP') 

# Decision Variables
x = mdl.binary_var_matrix(L, P, name='X')  # 1 if product p is assigned to slot l, 0 otherwise
y = mdl.integer_var_matrix(L, P, name='Y')  # Number of LC of product p in slot l

#CONSTRAINTS:

# 1. At most one product type per location
for l in range(L):
    mdl.add_constraint(
        mdl.sum(x[l,p] for p in range(P)) <= 1
    )

# 2. Slot capacity
for p in range(P):
    for l in range(L):
        mdl.add_constraint(
            VOL[p]*y[l,p]<=C*x[l,p]
        )

# 3. Total LCs allocation
for p in range(P):
    mdl.add_constraint(
        mdl.sum(y[l,p] for l in range(L)) == LC[p]
    )

# 4. Cast constraint to allocate only in Zone_10
for p in range(P):
    if cast[p] == 1:
        for zone in Stock_Zones:
            if zone != Zone_10:  # Exclude Zone_10 from the restriction
                mdl.add_constraint(
                    mdl.sum(x[l - 1, p] for l in zone) == 0  # Adjust index here
                )

# 5. Int/Ext constraint
for p in range(P):
    if internal[p] == 1:
        # Prevent allocation outside Zone_7, Zone_8, and Zone_9
        for zone in Stock_Zones:
            if zone not in (Zone_7, Zone_8, Zone_9):
                mdl.add_constraint(
                    mdl.sum(x[l-1, p] for l in zone) == 0  # No need for l-1
                )
        # Ensure allocation in Zone_7, Zone_8, or Zone_9
        mdl.add_constraint(
            mdl.sum(y[l, p] for l in Zone_7) + 
            mdl.sum(y[l, p] for l in Zone_8) + 
            mdl.sum(y[l, p] for l in Zone_9) == LC[p] 
        )

# 6. Force slots 11 and 12 to be occupied by A class products
for l in [10, 11]:  # Adjust for 0-based indexing
    mdl.add_constraint(
        mdl.sum(x[l, p] for p in range(P) if product_data['Class'][p] == 'A') == 1
    )


# OBJ FUNCTION
mdl.minimize(
    mdl.sum(dis[l]*x[l, p]*(3*A[p, 0] + 2*A[p, 1] + 1*A[p, 2]) for p in range(P) for l in range(L))
)

#print the implementation and solution of the objective function and the constraints
print(mdl.export_to_string())

# Set CPLEX parameters for detailed logging
mdl.parameters.mip.display = 2  # Display every node and integer solution
mdl.parameters.parallel = -1 # opportunistic
mdl.parameters.workmem = 16000 # 16 GB 
# Set CPLEX parameters for increased exploration
mdl.parameters.mip.strategy.search = 2  # Dynamic search
mdl.parameters.mip.limits.solutions = 56  # Explore more solutions
mdl.parameters.mip.strategy.nodeselect = 3  # Alternative best-estimate search
mdl.parameters.mip.strategy.variableselect = 1  # Strong branching
mdl.parameters.mip.strategy.backtrack = 0.1  # More exploration
mdl.parameters.mip.strategy.dive = 1  # Guided diving
mdl.parameters.randomseed = 42  # Set random seed

# warmstart=mdl.new_solution()
# warmstart.add_var_value(x,1)
# mdl.add_mip_start(warmstart)
mdl.parameters.timelimit = 1000
sol=mdl.solve(log_output=True)
sol.display()

#save the solution in a list
x_list = []
for l in range(L):
    tmp = False
    for p in range(P):
        if sol.get_value(x[l, p]) == 1:
            tmp = True 
            x_list.append([l+1, p+1])
    if not tmp:
        x_list.append([l+1, -1])

# 1. Write slot IDs to product_data(output).csv
product_data['Slot_IDs'] = ""  # Initialize an empty column for slot IDs
for l in range(L):
    for p in range(P):
        if sol.get_value(x[l, p]) == 1:
            product_data['Slot_IDs'][p] += f"{l+1}, "  # Add slot ID to the corresponding product row
product_data['Slot_IDs'] = product_data['Slot_IDs'].str.rstrip(', ')  # Remove trailing comma and space

# Add columns for each zone
for i, zone in enumerate(Stock_Zones):
    zone_name = f"Zone_{i+1}" 
    product_data[zone_name] = 0  # Initialize the column with 0

    for l in zone:
        for p in range(P):
            if sol.get_value(x[l-1, p]) == 1:
                product_data[zone_name][p] += int(sol.get_value(y[l-1, p]))


#save data frame product_data into a .csv
product_data.to_csv("./products_data(output).csv", index=True, header=True)

# 2. Create an array with product ID, slot ID, and #LC
product_allocation = []
for l in range(L):
    for p in range(P):
        if sol.get_value(x[l, p]) == 1:
            product_allocation.append([p+1, l+1, int(sol.get_value(y[l, p]))])

def sort_by_product_id_and_slot_id(allocation):
    return (allocation[0], allocation[1])

a_class_products = []
for allocation in product_allocation:
    p_id = allocation[0]
    if product_data['Class'][p_id-1] == 'A':
        a_class_products.append(allocation)

# Sort A class products by product ID and then slot ID
a_class_products.sort(key=sort_by_product_id_and_slot_id)

# Extract slot IDs (second column)
slot_ids = [allocation[1] for allocation in a_class_products]

# Sort slot IDs
slot_ids.sort()

for i in range(len(a_class_products)):
    a_class_products[i][1] = slot_ids[i]

# Create a dictionary mapping original slot IDs to sorted slot IDs
slot_mapping = dict(zip([allocation[1] for allocation in a_class_products], slot_ids))

# Reassign sorted slot IDs back to the array
for i in range(len(a_class_products)):
    a_class_products[i][1] = slot_mapping[a_class_products[i][1]]

a_prod=[]
for i in range(len(a_class_products)):
    original_product_index = a_class_products[i][1]  # Store original product index
    # Swap the first two columns (PRODUCT ID and SLOT ID)
    a_class_products[i][0], a_class_products[i][1] = a_class_products[i][1], a_class_products[i][0]
    # Extend a_prod with the individual elements from a_class_products[i] and the class
    a_prod.append(a_class_products[i] + [product_data['Class'][a_class_products[i][1]-1]]) 

# Visualization
fig = go.Figure()

# Add empty slots first
empty_slots = [l for l, p in x_list if p == -1]
empty_x = x_coord[np.array(empty_slots)-1]
empty_y = y_coord[np.array(empty_slots)-1]
empty_text = [f"Slot: {l}<br>Empty" for l in empty_slots]  # Corrected

fig.add_trace(go.Scatter(
    x=empty_x,
    y=empty_y,
    mode='markers',
    marker=dict(color='green', size=10),
    text=empty_text,
    hoverinfo='text',
    name='Unassigned'
))

# Add DEA_gate node to the graph
fig.add_trace(go.Scatter(
    x=[424],
    y=[26.155],
    mode='markers',
    marker=dict(color='red', size=10),
    text='DEA_gate',
    hoverinfo='text',
    name='DEA_gate'
))

# Add nodes in Z
# zx = [coord[0] for coord in Z]
# zy = [coord[1] for coord in Z]
# z_text = [f"Node: {i+1}" for i in range(len(Z))]

# fig.add_trace(go.Scatter(
#     x=zx,
#     y=zy,
#     mode='markers',
#     marker=dict(color='grey', size=10),
#     text=z_text,
#     hoverinfo='text',
#     name='Pickup_Nodes'
# ))

# Add slots for each product class with corresponding colors
for product_class, color in zip(['A', 'B', 'C'], ['blue', 'yellow', 'purple']):
    if product_class == 'A':  # Use a_class_products for class A
        occupied_slots = a_prod
    else:
        occupied_slots = [[l, p, product_data['Class'][p-1]] for l, p in x_list if p != -1 and product_data['Class'][p-1] == product_class]

    occupied_x = x_coord[np.array([slot[0] for slot in occupied_slots])-1]  # Extract slot IDs from occupied_slots
    occupied_y = y_coord[np.array([slot[0] for slot in occupied_slots])-1]

    # Include product type index in hover text
    occupied_text = [
        f"Slot: {slot[0]}<br>Class: {slot[3]}<br>Product Index: {slot[1]}<br>#LC: {slot[2]}" 
        if len(slot) == 4 else 
        f"Slot: {slot[0]}<br>Class: {slot[2]}<br>Product Index: {slot[1]}" 
        for slot in occupied_slots
    ]

    fig.add_trace(go.Scatter(
        x=occupied_x,
        y=occupied_y,
        mode='markers',
        marker=dict(color=color, size=10),
        text=occupied_text,
        hoverinfo='text',
        name=f'{product_class} Class'
    ))

# Update layout
fig.update_layout(
    title='MCV WH OPTIMIZATION',
    xaxis_title='X',
    yaxis_title='Y',
    hovermode='closest',
    showlegend=True
)

fig.show()

Invalid file path. Please try again.
Invalid file path. Please try again.
Invalid file path. Please try again.
Invalid file path. Please try again.
Invalid file path. Please try again.
Invalid file path. Please try again.
Invalid file path. Please try again.
Invalid file path. Please try again.
Invalid file path. Please try again.
Invalid file path. Please try again.
Invalid file path. Please try again.
